In [ ]:
class S_Dbw():
    def __init__(self,data,data_cluster,cluster_centroids_):
        """
        data --> raw data
        data_cluster --> The category that represents each piece of data(the number of category should begin 0)
        cluster_centroids_ --> the center_id of each cluster's center
        """
        self.data = data
        self.data_cluster = data_cluster
        self.cluster_centroids_ = cluster_centroids_
        self.k =35 ##only for reduced
        #self.k = cluster_centroids_.shape[0]#the number of labels
        self.stdev = 0                
        for i in range(self.k):
            if len(data[self.data_cluster == i])==0:
                break
            std_matrix_i = np.std(data[self.data_cluster == i],axis=0)
            self.stdev += np.sqrt(np.dot(std_matrix_i.T,std_matrix_i))
            print(i,self.stdev)
        self.stdev = np.sqrt(self.stdev)/(i+1) 
    def density(self,density_list=[]):
        """
        compute the density of one or two cluster(depend on density_list)
        
        """
        density = 0
        c=0
        for i in range(len(density_list)):
            c+=self.cluster_centroids_[density_list[i]]
        center_v=c/(i+1)
        for i in density_list:
            temp = self.data[self.data_cluster == i]
            for j in temp:    
                if np.linalg.norm(j - center_v,ord=np.inf) <= self.stdev:# infinite order
                    density += 1
        return density
    def Dens_bw(self):
        density_list = []
        result = 0
        for i in range(self.k):
            density_list.append(self.density(density_list=[i])) 
        
        for i in range(self.k):
            for j in range(self.k):
                if i==j:
                    continue
                if max(density_list[i],density_list[j])==0: #skip the extreme cluster
                    continue 
                result += self.density([i,j])/max(density_list[i],density_list[j])
        return result/(self.k*(self.k-1))
    def Scat(self):
        
        sigma_s = np.std(self.data,axis=0)
        sigma_s_2norm = np.sqrt(np.dot(sigma_s.T,sigma_s))

        
        sum_sigma_2norm = 0
        for i in range(self.k):
            matrix_data_i = self.data[self.data_cluster == i]
            sigma_i = np.std(matrix_data_i,axis=0)
            sum_sigma_2norm += np.sqrt(np.dot(sigma_i.T,sigma_i))
        return sum_sigma_2norm/(sigma_s_2norm*self.k)
    def S_Dbw_result(self):
        """
        compute the final result
        """
        return self.Dens_bw()+self.Scat()


In [ ]:
k_means_cluster_centers = df_cluster.iloc[:,1:].values#cluster centers
k_means_labels= df_en_abstract_only['K-Means_cluster_reduced'].values# cluster label
evaluation = S_Dbw(X_reduced,k_means_labels,k_means_cluster_centers)
print(evaluation.S_Dbw_result())